## Semantic text search using embeddings

We can search through all our reviews semantically in a very efficient manner and at very low cost, by embedding our search query, and then finding the most similar reviews. The dataset is created in the [Get_embeddings_from_dataset Notebook](Get_embeddings_from_dataset.ipynb).

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import ollama

datafile_path = "data/fine_food_reviews_with_embeddings_1k.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(literal_eval).apply(np.array)


Here we compare the cosine similarity of the embeddings of the query and the documents, and show top_n best matches.

In [2]:
from utils.embeddings_utils import cosine_similarity

def get_embedding(text: str, model="nomic-embed-text:latest"):
    response = ollama.embeddings(
      model=model,
      prompt=text,
    )
    return response["embedding"]

# search through the reviews for a specific product
def search_reviews(df, product_description, n=3, pprint=True):
    product_embedding = get_embedding(
        product_description,
        model="nomic-embed-text:latest"
    )
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .combined.str.replace("Title: ", "")
        .str.replace("; Content:", ": ")
    )
    if pprint:
        for r in results:
            print(r[:200])
            print()
    return results


results = search_reviews(df, "delicious beans", n=3)


Best beans your money can buy:  These are, hands down, the best jelly beans on the market.  There isn't a gross one in the bunch and each of them has an intense, delicious flavor.  Though I hesitate t

Jamaican Blue beans:  Excellent coffee bean for roasting. Our family just purchased another 5 pounds for more roasting. Plenty of flavor and mild on acidity when roasted to a dark brown bean and befor

Delicious!:  I enjoy this white beans seasoning, it gives a rich flavor to the beans I just love it, my mother in law didn't know about this Zatarain's brand and now she is traying different seasoning



In [3]:
results = search_reviews(df, "whole wheat pasta", n=3)


Rustichella ROCKS!:  Anything this company makes is worthwhile eating! My favorite is their Trenne.<br />Their whole wheat pasta is the best I have ever had.

sooo good:  tastes so good. Worth the money. My boyfriend hates wheat pasta and LOVES this. cooks fast tastes great.I love this brand and started buying more of their pastas. Bulk is best.

Tasty and Quick Pasta:  Barilla Whole Grain Fusilli with Vegetable Marinara is tasty and has an excellent chunky vegetable marinara.  I just wish there was more of it.  If you aren't starving or on a 



We can search through these reviews easily. To speed up computation, we can use a special algorithm, aimed at faster search through embeddings.

In [4]:
results = search_reviews(df, "bad delivery", n=1)


great product, poor delivery:  The coffee is excellent and I am a repeat buyer.  Problem this time was with the UPS delivery.  They left the box in front of my garage door in the middle of the drivewa



As we can see, this can immediately deliver a lot of value. In this example we show being able to quickly find the examples of delivery failures.

In [5]:
results = search_reviews(df, "spoilt", n=1)


Was rotten:  Culd not eat it, but the other flavor I ordered was fine, not sure why it was not ok, when the other one was



In [6]:
results = search_reviews(df, "pet food", n=2)


Healthy Dog Food:  This is a very healthy dog food. Good for their digestion. Also good for small puppies. My dog eats her required amount at every feeding.

Excellent Dog Food:  My dog loves it and she absorbes it well.  Her breath is fresh and her teeth are staying clean.  I free feed her and her weight gain is on schedule.

